In [1]:
from brix.grid_maker import Grid_maker
import geopandas as gpd
import json

In [2]:
s0_lu=gpd.read_file('../Data/GIS/Scenario0/Landuse_scenario_0/LandUse_0.shp')
s2_lu=gpd.read_file('../Data/GIS/Scenario2/Landuse_scenario_2/LandUse_2.shp')
s3_lu=gpd.read_file('../Data/GIS/Scenario3/Landuse_scenario_3/LandUse_3.shp')

In [3]:
def abbreviate_type_name(name):
    """
    Shorter type names for CityScope UI"""
    return name.split(',')[0].split(' - ')[0].split(' (')[0].replace('\n','').replace('\r','').strip()

#### Specify grid parameters

In [4]:
table_name='hcmc_rd'
top_left_lon=106.68353996851717
top_left_lat=10.772585487816258
rotation=-0.3151799175424706
cell_size=71 
ncols=66
nrows=37

#### Create grid object using cs-brix library

In [5]:
grid=Grid_maker(table_name=table_name, top_left_lon=top_left_lon, top_left_lat=top_left_lat, 
                     rotation=rotation, cell_size=cell_size, ncols=ncols, nrows=nrows)

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
Table hcmc_rd already exists
Calculating initial coordinates of each cell


/Users/doorleyr/anaconda3/envs/brix/lib/python3.9/site-packages/brix/grid_maker.py:82: UserWarning: Table hcmc_rd already exists
  warn(f'Table {self.table_name} already exists')


Defining properties and headers


#### Update default types to CSL_HCMC types

In [6]:
types=json.load(open('../Data/Table/types.json'))
grid.edit_types(types)

Generating grid geojson


#### Specify which cells are interactive
interactive.geojson is a rough outline of the nteractive region of the site, created using geojson.io. Should be updated if it is nt accurate enough

In [7]:
interactive=gpd.read_file('../Data/Table/interactive.geojson')
grid_gpd=gpd.read_file(json.dumps(grid.get_grid_geojson()))
interactive_over_grid=gpd.overlay(grid_gpd, interactive, how='intersection')
int_cell_ids=interactive_over_grid['id'].unique()
int_cell_ids

array([ 697,  698,  699,  762,  763,  764,  765,  766,  828,  829,  830,
        831,  832,  833,  895,  896,  897,  898,  899,  900,  961,  962,
        963,  964,  965,  966,  967, 1028, 1029, 1030, 1031, 1032, 1033,
       1034, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1163, 1164,
       1165, 1166, 1167, 1168, 1169, 1170, 1231, 1232, 1233, 1234, 1235,
       1236, 1237, 1238, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306,
       1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1434, 1435,
       1436, 1437, 1438, 1439, 1440, 1441, 1502, 1503, 1504, 1505, 1506,
       1507, 1570, 1571, 1572])

#### Create the three GEOGRID and GEOGRIDDATA files

In [8]:
lu_files={0: s0_lu, 2: s2_lu, 3: s3_lu}
scenario_grids={}

In [9]:
import copy
for s in lu_files:
    print(s)
    scenario_lu=lu_files[s]
    scenario_lu_wgs=scenario_lu.to_crs("EPSG:4326")
    grid_geojson=copy.deepcopy(grid.get_grid_geojson())
    grid_gpd=gpd.read_file(json.dumps(grid_geojson))
    lu_over_grid=gpd.overlay(scenario_lu_wgs, grid_gpd, 'intersection')   
    
    grid_to_type={}
    lu_over_grid['area']=lu_over_grid.geometry.area
    lu_over_grid=lu_over_grid.sort_values('area', ascending=True)
    for ind, row in lu_over_grid.iterrows():
        grid_to_type[row['id']]={'type': abbreviate_type_name(row['CSLLandTyp']), 'height': int(row['Storey'])}
        
    geogriddata=[]
    for i_c, cell in enumerate(grid_geojson['features']):
        assert i_c==cell['properties']['id']
        if cell['properties']['id'] not in grid_to_type:
            cs_type="None"
            height=0
        else:
            cs_type=grid_to_type[i_c]['type']
            height=grid_to_type[i_c]['height']
        if cell['properties']['id'] in int_cell_ids:
            is_interactive='Web'
        else:
            is_interactive=False
        color=types[cs_type]['color']
        if is_interactive==False:
            color[3]=100
        else:
            color[3]=200
        grid_geojson['features'][i_c]['properties']['interactive']=is_interactive
        grid_geojson['features'][i_c]['properties']['color']=color
        grid_geojson['features'][i_c]['properties']['name']=cs_type
        grid_geojson['features'][i_c]['properties']['height']=height

        geogriddata.append({'color': color, 'height': height, 'id': i_c, 'name': cs_type, 'interactive': is_interactive})
    scenario_grids[s]={'geogrid': grid_geojson, 'geogriddata': geogriddata}

0


<ipython-input-9-edd890829709>:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lu_over_grid['area']=lu_over_grid.geometry.area


2


<ipython-input-9-edd890829709>:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lu_over_grid['area']=lu_over_grid.geometry.area


3


<ipython-input-9-edd890829709>:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lu_over_grid['area']=lu_over_grid.geometry.area


#### Save the results

In [10]:
for s in scenario_grids:
    json.dump(scenario_grids[s]['geogrid'], open('results/scenario_{}/geogrid_scenario_{}.geojson'.format(s, s), 'w'))
    json.dump(scenario_grids[s]['geogriddata'], open('results/scenario_{}/geogrid_data_scenario_{}.json'.format(s, s), 'w'))